In [ ]:
# --- Step 1: Environment Setup & Data Ingestion for SINADEF ---
import pandas as pd
import numpy as np

print("Libraries imported successfully.")

# Define the path to the raw SINADEF Excel file
sinadef_excel_path = '../data/fallecidos_sinadef.xlsx'

try:
    # Load the Excel file. SINADEF datasets can be large, so this might take a moment.
    df_sinadef_raw = pd.read_excel(sinadef_excel_path, engine='openpyxl')
    print(f"Successfully loaded raw data from '{sinadef_excel_path}'.")
    print(f"Initial shape: {df_sinadef_raw.shape[0]} rows, {df_sinadef_raw.shape[1]} columns.")
    
    print("\n--- Initial Data Preview (First 5 Rows) ---")
    display(df_sinadef_raw.head())
    
    print("\n--- General DataFrame Info ---")
    df_sinadef_raw.info(verbose=True, show_counts=True)

except FileNotFoundError:
    print(f"CRITICAL ERROR: The file was not found at the specified path: {sinadef_excel_path}")

Libraries imported successfully.


In [ ]:
def process_sinadef_data(raw_df):
    """
    This function takes the raw SINADEF DataFrame, cleans it, and aggregates it
    by district ('ubigeo') to create the S-Factors for Project IRIS.
    """
    df = raw_df.copy()
    print("SINADEF processing pipeline initiated...")

    # --- Pasito 1: Standardize Column Names ---
    print("  -> Step 1: Standardizing column names to snake_case...")
    df.columns = (df.columns.str.lower()
                  .str.replace(' ', '_', regex=False)
                  .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

    # --- Pasito 2: Select and Rename Key Columns ---
    # We select only the columns we need to make the DataFrame lighter and more manageable.
    print("  -> Step 2: Selecting and renaming key columns...")
    # ASUNCIÓN: Estos son los nombres probables. ¡Tendremos que ajustarlos después de ver el .info()!
    columnas_de_interes = {
        'departamento_domicilio': 'departamento',
        'provincia_domicilio': 'provincia',
        'distrito_domicilio': 'distrito',
        'edad': 'edad',
        'sexo': 'sexo',
        # ASUNCIÓN: Necesitamos una columna con la causa de muerte, probablemente con códigos CIE-10
        'causa_cie_x': 'causa_cie10' 
    }
    # Filtramos para quedarnos solo con las columnas que existen en nuestro DataFrame
    columnas_a_usar = {k: v for k, v in columnas_de_interes.items() if k in df.columns}
    df = df[list(columnas_a_usar.keys())].rename(columns=columnas_a_usar)
    
    # --- Pasito 3: Shielding Text Data ---
    print("  -> Step 3: Shielding text columns (stripping whitespace)...")
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
        df[col] = df[col].str.strip()

    # --- Pasito 4: Create the UBIGEO Merge Key ---
    # This is the most critical step for data fusion.
    print("  -> Step 4: Engineering the 'ubigeo' merge key...")
    df['ubigeo'] = df['departamento'].astype(str) + '_' + df['provincia'].astype(str) + '_' + df['distrito'].astype(str)
    
    # --- Pasito 5: Feature Engineering (S-Factors) ---
    print("  -> Step 5: Engineering the S-Factor metrics...")
    
    # S-Factor 1: Tasa de Causas Prevenibles
    # INVESTIGACIÓN REQUERIDA: Necesitamos una lista de códigos CIE-10 para causas de muerte prevenibles.
    # Este es un ejemplo. La lista real debe ser investigada y validada.
    lista_codigos_prevenibles = ['A0', 'A1', 'A3', 'B15', 'B16', 'B17', 'B20', 'E4'] # Ejemplos: Cólera, TBC, Sarampión, Hepatitis, VIH, Desnutrición
    
    if 'causa_cie10' in df.columns:
        # Creamos una bandera si el código de muerte empieza con alguno de los códigos prevenibles.
        df['es_causa_prevenible'] = df['causa_cie10'].astype(str).str.startswith(tuple(lista_codigos_prevenibles)).astype(int)
    else:
        df['es_causa_prevenible'] = 0 # Si no hay datos de causa, asumimos 0.
        print("     - WARNING: 'causa_cie10' column not found. Tasa de causas prevenibles will be 0.")

    # --- Pasito 6: Aggregation by District ---
    print("  -> Step 6: Aggregating data by district...")
    df_aggregated = df.groupby('ubigeo').agg(
        # Contamos el total de muertes usando el índice.
        s_factor_total_muertes=('edad', 'count'),
        # Calculamos la edad promedio de muerte.
        s_factor_edad_prom_muerte=('edad', 'mean'),
        # La media de una columna 0/1 es el porcentaje (tasa).
        s_factor_tasa_prevenibles=('es_causa_prevenible', 'mean')
    ).reset_index()

    print("\nSINADEF processing and aggregation complete!")
    return df_aggregated